CLEANING LIVE RAW DATA AND CONVERTING IT TO APPROPRIATE DATA TYPES

In [9]:
#Imports

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
import os


In [10]:
#Spark Session Initialization

spark = SparkSession.builder \
    .appName("clean_train_live_status") \
    .config("spark.jars.packages","io.delta:delta-spark_2.13:4.0.0") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.properties.defaults.enableDeletionVectors", "true") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "20") \
    .getOrCreate()


current_date = datetime.now().date()

In [ ]:
# Data Load

df = spark.read.format("delta")\
    .load(os.path.join(os.path.abspath(""),f"raw_data\\raw_live_data\\{current_date}\\trainlivestatus"))

In [ ]:
# casting to appropriate types

cleaned_df = df.withColumn("createdDate",col("createdDate").cast(TimestampType()))\
    .withColumn("distanceCovered",col("distanceCovered").cast(IntegerType()))\
    .withColumns({"totalDistance":col("totalDistance").cast(IntegerType()),
                  "noOfDays":col("noOfDays").cast(IntegerType())})


root
 |-- eventId: string (nullable = true)
 |-- trainId: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- stationCode: string (nullable = true)
 |-- distanceCovered: integer (nullable = true)
 |-- totalDistance: integer (nullable = true)
 |-- noOfDays: integer (nullable = true)
 |-- actualArrivalTime: string (nullable = true)
 |-- expectedArrivalTime: string (nullable = true)
 |-- actualDepartureTime: string (nullable = true)
 |-- expectedDepartureTime: string (nullable = true)
 |-- createdDate: timestamp (nullable = true)



In [100]:
# replacing NAN values in time columns with "00:00"

# to_replace = is value to be replaced
# value = is the value to replace with
# subste = is the columns list to apply the replacement

cleaned_new_df = cleaned_df\
    .na.replace(to_replace=['"NaN"',"NaN","Nan","nan",""], value="00:00",subset=["actualArrivalTime","expectedArrivalTime"])


In [ ]:
df.write.format("delta") \
    .mode("append") \
    .partitionBy("createdDate") \
    .save(os.path.join(os.path.abspath(""),f"silver_data\\live_data\\trainlivestatus_cleaned"))